# ДЗ Неделя 2 — GRPO обучение (TRAIN)

Этот ноутбук предназначен **только для обучения** (Unsloth + TRL/GRPO) и сохранения обученной модели.

Что вы получите на выходе:
- `models/<cfg.merged_model_subdir>/` — **merged** веса (base + LoRA), готовые для vLLM/transformers
- `results/trained_model.json` — путь к merged‑модели
- `data/test_*.jsonl` и `data/dev_*.jsonl` — фиксированные наборы задач (не пересэмпливаются)

Для подсчёта метрик **до/после** обучения используйте отдельный ноутбук:
`week2_grpo_lis_infer_eval.ipynb` (желательно в **отдельной среде** с `vLLM`).

---


## Важно про `torch.compile` / Unsloth

Если во время обучения вы видите ошибку вида `FailOnRecompileLimitHit: recompile_limit reached with fullgraph=True`,
это значит, что `torch.compile` слишком часто перекомпилирует граф (обычно из‑за разных длин последовательностей).

В этом ноутбуке есть 2 рычага в `ExperimentConfig`:
- `unsloth_compile_disable=True` — полностью отключить auto‑compiler в Unsloth (стабильнее, но медленнее).
- `torchdynamo_cache_size_limit` / `torchdynamo_accumulated_cache_size_limit` — увеличить лимиты кеша TorchDynamo,
  чтобы fullgraph‑компиляция не падала.

**Если вы меняете `unsloth_compile_disable`, лучше сделать Restart Kernel и запустить ноутбук заново**, чтобы флаг
успел примениться до импорта `unsloth`.


## 0. Подготовка окружения

Если запускаете в **Google Colab** / **Kaggle**, включите GPU:

- Colab: *Runtime → Change runtime type → GPU*
- Kaggle: *Settings → Accelerator → GPU*

Затем выполните ячейку с установкой зависимостей ниже.


### 0.1 Конфигурация эксперимента (одна точка правды)

Чтобы затем **без боли перенести код в репозиторий**, держим все важные параметры (модель, датасеты, GRPO, пути, seed) в одном объекте `cfg`.

Дальше по ноутбуку мы стараемся **не использовать “магические числа”**, а брать значения из `cfg`.


In [1]:
import os
import re
import json
import math
import random
from dataclasses import dataclass, asdict, field
from typing import Optional, List, Dict, Any, Iterable

from abc import ABC, abstractmethod
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# --- Конфиг эксперимента ---
from dataclasses import dataclass, field, asdict
from typing import Any, Dict, List, Optional
from pathlib import Path
import json, random
import numpy as np

@dataclass
class ExperimentConfig:
    # ====== Модель ======
    base_model: str = "Qwen/Qwen2.5-1.5B-Instruct"

    # ====== Воспроизводимость ======
    global_seed: int = 42

    # ====== Пути ======
    data_dir: str = "data"
    results_dir: str = "results"
    models_dir: str = "models"

    # ====== Эксперимент‑трекинг (Comet ML) ======
    comet_enabled: bool = True
    comet_project_name: str = "ds-mag-w2-grpo"
    comet_workspace: Optional[str] = None
    comet_offline: bool = False
    comet_tags: List[str] = field(default_factory=lambda: ["W2", "GRPO", "LIS"])

    # ====== Фиксированные тестовые наборы (для финальной оценки в eval-env) ======
    recreate_tests: bool = False
    test_specs: List[Dict[str, Any]] = field(default_factory=lambda: [
        {"name": "easy",   "difficulty": 2, "n": 200, "seed": 1001},
        {"name": "medium", "difficulty": 5, "n": 200, "seed": 2001},
        {"name": "hard",   "difficulty": 8, "n": 200, "seed": 3001},
    ])

    # ====== Dev-наборы (валидация во время RL) ======
    # Делаем НЕ один dev, а 3 "среза" по сложности, чтобы видеть:
    # - не забываем ли простые примеры,
    # - растём ли на сложных.
    recreate_dev: bool = False
    dev_specs: List[Dict[str, Any]] = field(default_factory=lambda: [
        {"name": "easy",   "difficulty": 2, "n": 128, "seed": 4001},
        {"name": "medium", "difficulty": 5, "n": 128, "seed": 4002},
        {"name": "hard",   "difficulty": 8, "n": 128, "seed": 4003},
    ])

    # Быстрый dev‑eval: на каждом прогоне берём поднабор (по каждому devset)
    dev_eval_n: int = 32
    dev_eval_every_steps: int = 50

    # Параметры генерации в dev-eval
    dev_max_new_tokens: int = 64
    dev_temperature: float = 0.0  # 0 -> greedy
    dev_full_eval_on_best: bool = True

    # Каким dev‑метриком выбираем "best checkpoint"
    # Важно: сравниваем correctness/dev accuracy, а не "reward".
    best_metric: str = "dev/avg"  # варианты: "dev/avg", "dev/hard", "dev/medium", "dev/easy"
    dev_metric_weights: Dict[str, float] = field(default_factory=lambda: {
        "easy": 0.25,
        "medium": 0.25,
        "hard": 0.50,
    })
    best_min_delta: float = 1e-6

    # Early‑stop по dev (чтобы не "разгонять" фазу до деградации)
    early_stop_enabled: bool = False
    early_stop_patience_evals: int = 8
    early_stop_warmup_evals: int = 2  # первые N eval-ов не останавливаем

    # ====== Train-набор (предсэмплированный) ======
    recreate_train: bool = False
    train_size: int = 8000
    train_min_difficulty: int = 1
    train_max_difficulty: int = 10
    train_seed: int = 2025

    # ====== Curriculum по сложности ======
    curriculum_enabled: bool = True
    start_phase_idx: int = 0  # можно поставить 1, чтобы запускать только фазу2 (при наличии сохранённого best фазы1)
    curriculum_phases: List[Dict[str, Any]] = field(default_factory=lambda: [
        {
            "name": "phase1_d1-5",
            "min_difficulty": 1,
            "max_difficulty": 5,
            "train_size": 8000,
            "steps": 200,
            "seed_offset": 0,
            "learning_rate": 1e-5,
            "kl_beta": 0.02,
            "dev_eval_every_steps": 50,
            "dev_eval_n": 32,
            "max_completion_length": 64,
            # на фазе1 weights можно сместить на medium (чтобы стабилизировать базовую способность)
            "best_metric": "dev/avg",
            "dev_metric_weights": {"easy": 0.30, "medium": 0.60, "hard": 0.10},
            "early_stop_patience_evals": 8,
        },
        {
            "name": "phase2_d6-10",
            "min_difficulty": 6,
            "max_difficulty": 10,
            "train_size": 1200,  # чуть больше данных, чтобы меньше переобучаться на "хард"
            "steps": 400,
            "seed_offset": 1,
            "learning_rate": 5e-6,  # осторожнее
            "kl_beta": 0.04,        # сильнее держим KL
            "dev_eval_every_steps": 20,
            "dev_eval_n": 32,
            "max_completion_length": 32,  # LIS ответы короткие -> меньше мусора и меньше шанс рекомпиляций
            # ВАЖНО: replay простых примеров, чтобы не было forgetting'а
            "replay_fraction": 0.20,
            "replay_min_difficulty": 4,
            "replay_max_difficulty": 5,
            # на фазе2 больше весим hard
            "best_metric": "dev/avg",
            "dev_metric_weights": {"easy": 0.20, "medium": 0.20, "hard": 0.60},
            "early_stop_patience_evals": 4,
        },
    ])

    # ====== Unsloth / модель ======
    max_seq_length: int = 2048
    load_in_4bit: bool = True

    # ====== LoRA ======
    lora_r: int = 16
    lora_alpha: int = 16
    lora_dropout: float = 0.0

    # ====== Reward shaping ======
    format_penalty: float = -0.1
    distance_reward_weight: float = 0.05

    # ====== GRPO ======
    grpo_output_dir: str = "grpo_lis_qwen2p5_1p5b"

    learning_rate: float = 1e-5
    per_device_train_batch_size: int = 1
    gradient_accumulation_steps: int = 8
    num_generations: int = 8
    max_prompt_length: int = 512
    max_completion_length: int = 64

    stop_on_answer_tag: bool = True
    stop_string: str = "</answer>"

    kl_beta: float = 0.02
    logging_steps: int = 10
    save_steps: int = 200
    # периодические чекпоинты Trainer (для resume_from_checkpoint)
    save_total_limit: Optional[int] = None  # None -> не удалять старые чекпоинты
    resume_from_checkpoint: Optional[str] = None  # путь к checkpoint-* для продолжения обучения
    max_steps: int = 800

    # ====== Стабильность (убираем torch.compile, если он ломает обучение) ======
    disable_unsloth_compile: bool = True   # UNSLOTH_COMPILE_DISABLE=1
    disable_torch_compile: bool = True    # torch.compile -> identity
    disable_torchdynamo: bool = True      # torch._dynamo.config.disable=True
    torchdynamo_recompile_limit: int = 256

    # ====== Чекпоинты фаз ======
    resume_from_previous_phase_best: bool = True
    save_phase_best: bool = True
    phase_checkpoints_dir: str = "results/phase_checkpoints"

    # ====== Выбор лучшей модели по dev ======
    select_best_by_dev: bool = True
    save_last_merged: bool = False
    merged_model_subdir: str = "qwen2p5_1p5b_grpo_lis_merged"


cfg = ExperimentConfig()

# --- Применяем seed ---
GLOBAL_SEED = cfg.global_seed
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

# --- Папки проекта ---
DATA_DIR = Path(cfg.data_dir)
RESULTS_DIR = Path(cfg.results_dir)
MODELS_DIR = Path(cfg.models_dir)
for p in [DATA_DIR, RESULTS_DIR, MODELS_DIR]:
    p.mkdir(parents=True, exist_ok=True)

def save_json(obj: Any, path: Path) -> None:
    """Утилита для сохранения JSON с нормальными отступами."""
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)

# Сохраняем конфиг, чтобы эксперимент было проще воспроизвести
save_json(asdict(cfg), RESULTS_DIR / "config.json")

# Диагностика
eff_batch = cfg.per_device_train_batch_size * cfg.gradient_accumulation_steps
print("Конфиг сохранён в:", RESULTS_DIR / "config.json")
print("DATA_DIR    =", DATA_DIR.resolve())
print("RESULTS_DIR =", RESULTS_DIR.resolve())
print("MODELS_DIR  =", MODELS_DIR.resolve())
print(f"Эффективный batch (prompts/step) = {eff_batch}  (bs={cfg.per_device_train_batch_size} * accum={cfg.gradient_accumulation_steps})")
print(f"num_generations (G) = {cfg.num_generations}  -> completions/step ~ {eff_batch * cfg.num_generations}")


Конфиг сохранён в: results/config.json
DATA_DIR    = /home/yaros/DS-Mag/AI-SelectedTopics/W2-1/data
RESULTS_DIR = /home/yaros/DS-Mag/AI-SelectedTopics/W2-1/results
MODELS_DIR  = /home/yaros/DS-Mag/AI-SelectedTopics/W2-1/models
Эффективный batch (prompts/step) = 8  (bs=1 * accum=8)
num_generations (G) = 8  -> completions/step ~ 64


### 0.2 Comet ML (логирование метрик и графиков)

Если хотите видеть **графики reward / KL / accuracy / длины ответов** и сохранять артефакты эксперимента в Comet:

1) Установите пакет:
```bash
pip install -U comet_ml
```

2) Задайте ключ (и при необходимости workspace):
```bash
export COMET_API_KEY="ВАШ_КЛЮЧ"
export COMET_WORKSPACE="ваш-workspace"   # опционально
```

3) Включите логирование:
```python
cfg.comet_enabled = True
```

Если нет доступа к интернету, можно включить `cfg.comet_offline = True` (тогда логи сохранятся локально).


In [3]:
def init_comet(cfg: ExperimentConfig):
    """Инициализируем Comet Experiment (если включено).

    Возвращает объект experiment или None (если Comet отключён/не настроен).
    """
    if not cfg.comet_enabled:
        print("[COMET] disabled (cfg.comet_enabled=False)")
        return None

    try:
        from comet_ml import Experiment, OfflineExperiment
    except Exception as e:
        print("[COMET] comet_ml не установлен или не импортируется:", repr(e))
        return None

    workspace = cfg.comet_workspace or os.getenv("COMET_WORKSPACE")
    project_name = cfg.comet_project_name or os.getenv("COMET_PROJECT_NAME") or "ds-mag-w2-grpo"

    if cfg.comet_offline:
        exp = OfflineExperiment(project_name=project_name, workspace=workspace)
        print("[COMET] OfflineExperiment создан:", project_name)
    else:
        api_key = "I3UVgV6rXpj7bzgPpAH2mrTcX"
        if not api_key:
            print("[COMET] COMET_API_KEY не задан -> Comet выключен. (export COMET_API_KEY=...)")
            return None
        exp = Experiment(api_key=api_key, project_name=project_name, workspace=workspace)
        print("[COMET] Experiment создан:", project_name)

    run_name = f"{cfg.base_model.split('/')[-1]}_seed{cfg.global_seed}"
    try:
        exp.set_name(run_name)
    except Exception:
        pass

    try:
        exp.log_parameters(asdict(cfg))
    except Exception:
        pass

    if cfg.comet_tags:
        try:
            exp.add_tags(cfg.comet_tags)
        except Exception:
            pass

    # Немного информации о среде
    try:
        import torch
        exp.log_other("torch_version", torch.__version__)
        exp.log_other("torch_cuda", str(torch.version.cuda))
        if torch.cuda.is_available():
            exp.log_other("gpu", torch.cuda.get_device_name(0))
    except Exception:
        pass

    return exp

COMET_EXPERIMENT = init_comet(cfg)


COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/yaroslav-pankratov/ds-mag-w2-grpo/58621e6aa3854b9d8267f0cf7a15daef

COMET INFO: Couldn't find a Git repository in '/home/yaros/DS-Mag/AI-SelectedTopics/W2-1' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


[COMET] Experiment создан: ds-mag-w2-grpo


## 1. Базовые интерфейсы (`Env`, `Verifier`, `Data`)

Ниже — минимальные интерфейсы из спецификации ДЗ.  
Для простоты держим их прямо в ноутбуке, но в репозитории обычно выносят в `base/`.


In [4]:
# --- Среда/верификатор вынесены в отдельный модуль (см. lis_env.py) ---
import sys
from pathlib import Path

PROJECT_DIR = Path(".").resolve()
if str(PROJECT_DIR) not in sys.path:
    sys.path.insert(0, str(PROJECT_DIR))

from lis_env import Data, Verifier, Env, LISVerifier, LISEnv

# Создаём среду
env = LISEnv()


## 2. Шаблон промпта (в отдельном файле)

По условиям ДЗ нужно сделать отдельный файл с функцией, которая добавляет правила задачи **на английском**.  
Для удобства мы создаём `prompt_templates.py` прямо из ноутбука.


In [5]:
# prompt_templates.py хранит правила игры/формат задания (в репозитории рядом с ноутбуком)
from prompt_templates import build_lis_prompt

# sanity-check импорта
_ = build_lis_prompt([3, 1, 2])


## 3. Среда: длина LIS

Реализуем:

- `LISEnv.generate(...)`: сэмплирует последовательности и считает *gold* длину LIS.
- `LISVerifier.extract_answer(...)`: извлекает финальный ответ из вывода модели.
- `LISVerifier.verify(...)`: сравнивает извлечённый ответ с *gold* ответом.

Управление сложностью:

- чем выше `difficulty`, тем длиннее последовательность и шире диапазон значений;
- дополнительно отбрасываем тривиальные случаи, где LIS равна 1 или равна длине последовательности.


In [6]:
# Реализация LIS-среды и верификатора находится в lis_env.py.
# Этот ноутбук использует готовый модуль через импорт выше.


## 4. Быстрая проверка: генерация + верификация

Быстрая проверка, что:

- `generate(...)` действительно создаёт корректные задачи/ответы;
- `extract_answer(...)` правильно достаёт число из формата `<answer>...</answer>`;
- `verify(...)` возвращает `True` для правильного и `False` для неправильного решения.


In [7]:
sample = env.generate(num_of_questions=3, difficulty=3, seed=123)
for i, d in enumerate(sample, 1):
    print(f"--- Задание {i} (сложность={d.difficulty}) ---")
    print(d.question)
    print("ответ:", d.answer)

# имитируем ответ модели в требуемом формате
fake_solution_ok = """<think>какие-то рассуждения</think>
<answer> {}</answer>""".format(sample[0].answer)

fake_solution_bad = """<think>...</think>
<answer> 999</answer>"""

print("\nВерификатор OK:", env.verify(sample[0], fake_solution_ok))
print("Верификатор BAD:", env.verify(sample[0], fake_solution_bad))


--- Задание 1 (сложность=3) ---
You are given a sequence of integers.

Task: compute the length of the Longest Increasing Subsequence (LIS).

Definitions:
- A subsequence is obtained by deleting zero or more elements without changing the order of remaining elements.
- An increasing subsequence is STRICTLY increasing: each next element is > the previous one.

Output requirements:
- Return ONLY the LIS length as an integer.

Sequence: [-13, -6, -12, 10, -1, -6, -11, 12, 14, 13, -13, -2]

ответ: 5
--- Задание 2 (сложность=3) ---
You are given a sequence of integers.

Task: compute the length of the Longest Increasing Subsequence (LIS).

Definitions:
- A subsequence is obtained by deleting zero or more elements without changing the order of remaining elements.
- An increasing subsequence is STRICTLY increasing: each next element is > the previous one.

Output requirements:
- Return ONLY the LIS length as an integer.

Sequence: [3, 3, -4, -4, 13, -13, -9, -10, -4, 3, -4, 8]

ответ: 5
--- За

## 5. Датасеты

Требования ДЗ:

- **Обучающий датасет** может быть сэмплируемым/итеративным (идея: «таски приходят потоком»).
- **Тестовые датасеты** должны быть фиксированными: генерируются один раз и дальше не меняются (для воспроизводимости).

Практический момент:

- В некоторых версиях `GRPOTrainer` **не поддерживает** `IterableDataset` и падает с `NotImplementedError`.
  Поэтому мы делаем «взрослый компромисс»: используем генератор задач, но **предсэмплируем**
  конечный train set фиксированного размера (map-style `datasets.Dataset`) и сохраняем его на диск.

Сделаем:

- 3 фиксированных тестовых набора: `easy / medium / hard`;
- 1 фиксированный dev-набор (маленький) для контроля качества в процессе RL;
- генератор задач + предсэмплинг train-набора.


In [8]:
# -------------------------
# Фиксированные наборы данных на диске (test + dev)
# -------------------------

from pathlib import Path
from typing import List, Dict, Any

from w2_utils import save_jsonl, load_jsonl

def make_or_load_fixed_testset(
    env: Any,
    n: int,
    difficulty: int,
    seed: int,
    out_path: Path,
    recreate: bool,
) -> List[Dict[str, Any]]:
    """Создаём/загружаем фиксированный набор для eval (prompt=вопрос, answer=gold)."""
    if out_path.exists() and (not recreate):
        return load_jsonl(out_path)

    rng = random.Random(int(seed))
    rows: List[Dict[str, Any]] = []
    for _ in range(int(n)):
        # sample_task есть в lis_env.Env (удобный шорткат над generate(num_of_questions=1))
        item = env.sample_task(rng=rng, difficulty=int(difficulty))

        # поддерживаем разные форматы (dict/Data)
        if isinstance(item, dict):
            q = item.get("question") or item.get("prompt") or ""
            a = item.get("answer") or ""
        else:
            q = getattr(item, "question", "") or getattr(item, "prompt", "")
            a = getattr(item, "answer", "")

        rows.append({"prompt": str(q), "answer": str(a)})

    save_jsonl(rows, out_path)
    return rows


# --- test sets (для финальной оценки в infer/eval ноутбуке) ---
fixed_testsets: Dict[str, List[Dict[str, Any]]] = {}
for spec in cfg.test_specs:
    p = DATA_DIR / f"test_{spec['name']}_d{spec['difficulty']}_n{spec['n']}_seed{spec['seed']}.jsonl"
    fixed_testsets[spec["name"]] = make_or_load_fixed_testset(
        env=env,
        n=int(spec["n"]),
        difficulty=int(spec["difficulty"]),
        seed=int(spec["seed"]),
        out_path=p,
        recreate=cfg.recreate_tests,
    )

# --- dev sets (для валидации во время RL) ---
fixed_devsets: Dict[str, List[Dict[str, Any]]] = {}
for spec in cfg.dev_specs:
    p = DATA_DIR / f"dev_{spec['name']}_d{spec['difficulty']}_n{spec['n']}_seed{spec['seed']}.jsonl"
    fixed_devsets[spec["name"]] = make_or_load_fixed_testset(
        env=env,
        n=int(spec["n"]),
        difficulty=int(spec["difficulty"]),
        seed=int(spec["seed"]),
        out_path=p,
        recreate=cfg.recreate_dev,
    )

# удобный алиас на средний dev (если где-то нужен один dev)
fixed_devset = fixed_devsets.get("medium") or next(iter(fixed_devsets.values()))

print("Fixed testsets:", {k: len(v) for k,v in fixed_testsets.items()})
print("Fixed devsets:",  {k: len(v) for k,v in fixed_devsets.items()})


Fixed testsets: {'easy': 200, 'medium': 200, 'hard': 200}
Fixed devsets: {'easy': 128, 'medium': 128, 'hard': 128}


### 5.1 Генератор задач и предсэмплинг train-набора

Мы генерируем задачи «на лету» бесконечным генератором, но **перед обучением** берем из него `N` примеров
и превращаем их в обычный `datasets.Dataset`.

Так обучение становится:

- воспроизводимым (один и тот же `seed` → один и тот же train set),
- переносимым в репозиторий (можно хранить train.jsonl как артефакт),
- совместимым с текущей реализацией `GRPOTrainer`.


In [9]:
def training_generator(env: Env, min_difficulty: int = 1, max_difficulty: int = 10, seed: int = 1234):
    """Бесконечный генератор задач для обучения.

    Важно: GRPOTrainer в некоторых версиях НЕ поддерживает IterableDataset.
    Поэтому на практике мы используем этот генератор, чтобы *предсэмплировать* конечный train set.
    """
    rng = random.Random(seed)
    while True:
        d = rng.randint(min_difficulty, max_difficulty)
        task = env.generate(num_of_questions=1, difficulty=d, seed=rng.randint(0, 10_000_000))[0]
        yield {
            "question": task.question,
            "answer": task.answer,
            "difficulty": task.difficulty,
            "metadata": task.metadata,
        }

def sample_train_stream(env: Env, n: int, min_difficulty: int, max_difficulty: int, seed: int):
    """Берём n элементов из training_generator (удобно для предсэмплинга)."""
    gen = training_generator(env, min_difficulty=min_difficulty, max_difficulty=max_difficulty, seed=seed)
    return [next(gen) for _ in range(n)]

# Посмотрим несколько примеров
samples = sample_train_stream(
    env,
    n=3,
    min_difficulty=cfg.train_min_difficulty,
    max_difficulty=cfg.train_max_difficulty,
    seed=cfg.train_seed,
)
for s in samples:
    print(s["difficulty"], s["answer"], s["question"].splitlines()[-1])


9 9 Sequence: [0, -31, 28, 6, 20, 13, -1, -4, -8, -28, -22, -13, 29, -12, -27, 32, -4, -8, 25, -1, -20, 28, 0, 16]
8 7 Sequence: [0, 12, 4, -15, 0, -21, -28, 6, -10, 25, 28, -10, 0, 1, 2, -12, 22, -4, 24, 6, -28, 1]
9 6 Sequence: [-3, 16, 19, 7, -11, 1, 21, -28, 32, -9, 32, 22, 27, -28, 0, -14, 14, 12, 18, 10, 7, -11, -13, 10]


## 6. System prompt и извлечение ответа

По условиям ДЗ нужно использовать system prompt, который требует формат:

```
<think> ... </think>
<answer> ... </answer>
```

Верификатор извлекает число из `<answer> ... </answer>` (запасной вариант: последнее целое число в тексте).


In [10]:
from w2_utils import SYSTEM_PROMPT


## 7. Вспомогательные функции для оценки (предпочтительно vLLM)

Точность считаем как:

\[
\text{accuracy} = \frac{\#\text{correct}}{\#\text{total}}
\]

Код ниже:

- собирает chat-prompts через `tokenizer.apply_chat_template(...)`;
- делает инференс через **vLLM** (если установлен), иначе использует `transformers.generate()` как fallback.


In [11]:
from w2_utils import build_chat_prompt, cleanup_cuda


## 9. GRPO-обучение с Unsloth

Дальше:

- загружаем базовую модель через `FastLanguageModel`;
- подключаем LoRA-адаптеры;
- создаём итеративный датасет промптов;
- определяем `correctness_reward_func` как обёртку над `env.verify`;
- обучаем через `GRPOTrainer`.

> Примечание: подстройте размер батча / число шагов под ваш GPU.


In [12]:
def run_train(cfg: ExperimentConfig, env, experiment=None) -> str:
    """
    GRPO-обучение (curriculum по фазам) + dev-eval + сохранение best LoRA по каждой фазе и глобального best.

    Ключевые идеи для стабильной фазы 2:
      - replay простых примеров в phase2 (чтобы не было forgetting)
      - несколько dev-срезов (easy/medium/hard) и best по взвешенному avg
      - max_completion_length поменьше в phase2
      - early-stop по dev (чтобы не "дожимать" до деградации)
      - жёстко отключаем torch.compile/TorchDynamo, если ловите FailOnRecompileLimitHit
    """
    import os, sys, shutil, math
    from pathlib import Path
    import random
    import numpy as np
    import torch
    from typing import Any, Dict, List, Optional

    experiment = experiment or globals().get("COMET_EXPERIMENT", None)

    # -------------------------
    # 0) Стабилизация (до импорта unsloth)
    # -------------------------
    if cfg.disable_unsloth_compile:
        os.environ["UNSLOTH_COMPILE_DISABLE"] = "1"
    # ВАЖНО: fullgraph=False (иначе шанс FailOnRecompileLimitHit гораздо выше)
    os.environ["UNSLOTH_FULLGRAPH"] = "0"
    os.environ["UNSLOTH_COMPILE_IGNORE_ERRORS"] = "1"

    if cfg.disable_torchdynamo:
        os.environ["TORCHDYNAMO_DISABLE"] = "1"
    if cfg.disable_torch_compile:
        os.environ["TORCH_COMPILE_DISABLE"] = "1"

    # Снести автосгенерённый кэш Unsloth (чтобы не подхватить старые артефакты)
    shutil.rmtree(Path.cwd() / "unsloth_compiled_cache", ignore_errors=True)

    # Runtime-переключатели (на случай если torch уже импортировали раньше)
    try:
        import torch._dynamo
        if cfg.disable_torchdynamo:
            torch._dynamo.config.disable = True
        torch._dynamo.config.suppress_errors = True
        if hasattr(torch._dynamo.config, "recompile_limit"):
            torch._dynamo.config.recompile_limit = int(cfg.torchdynamo_recompile_limit)
    except Exception:
        pass

    if cfg.disable_torch_compile and hasattr(torch, "compile"):
        import torch

        def _noop_torch_compile(fn=None, *args, **kwargs):
            # Вариант 1: torch.compile(fn, ...)
            if callable(fn):
                return fn
            # Вариант 2: torch.compile(...)-> decorator
            def _decorator(f):
                return f
            return _decorator

        # Патчим
        torch.compile = _noop_torch_compile

    # Если Unsloth уже был импортирован ДО установки env, лучше перезапустить kernel
    if "unsloth" in sys.modules:
        print("[WARN] `unsloth` уже импортирован в этом kernel'е. "
              "Переменные среды UNSLOTH_* могли НЕ примениться. "
              "Если ловите FailOnRecompileLimitHit — сделайте Restart Kernel и Run All.")

    # -------------------------
    # 1) Импортируем нужное для обучения
    # -------------------------
    from unsloth import FastLanguageModel
    from trl import GRPOConfig, GRPOTrainer
    from datasets import Dataset
    import inspect
    import re

    # -------------------------
    # 2) Dev-наборы: готовим заранее (фиксация + воспроизводимость)
    # -------------------------
    def _dev_path(spec: Dict[str, Any]) -> Path:
        return DATA_DIR / f"dev_{spec['name']}_d{spec['difficulty']}_n{spec['n']}_seed{spec['seed']}.jsonl"

    dev_sets: Dict[str, List[Dict[str, Any]]] = {}
    for spec in cfg.dev_specs:
        name = str(spec["name"])
        path = _dev_path(spec)
        ds = make_or_load_fixed_testset(
            env=env,
            out_path=path,
            n=int(spec["n"]),
            difficulty=int(spec["difficulty"]),
            seed=int(spec["seed"]),
            recreate=cfg.recreate_dev,
        )
        dev_sets[name] = ds

    for name, rows in dev_sets.items():
        for r in rows:
            if "prompt" not in r or not r["prompt"]:
                q = r.get("question")
                if q is None:
                    continue
                r["prompt"] = q  # именно сырой question; chat prompt строим позже через build_chat_prompt

    # Детерминированные индексы для quick dev-eval
    dev_quick_indices: Dict[str, List[int]] = {}
    for spec in cfg.dev_specs:
        name = str(spec["name"])
        n = len(dev_sets[name])
        k = min(cfg.dev_eval_n, n)
        rng = random.Random(cfg.global_seed + 999 + int(spec["seed"]))
        dev_quick_indices[name] = rng.sample(range(n), k)

    # -------------------------
    # 3) Загружаем базовую модель + LoRA
    # -------------------------
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=cfg.base_model,
        max_seq_length=cfg.max_seq_length,
        dtype=None,
        load_in_4bit=cfg.load_in_4bit,
        use_gradient_checkpointing=False,  # стабильнее
    )

    model = FastLanguageModel.get_peft_model(
        model,
        r=cfg.lora_r,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                        "gate_proj", "up_proj", "down_proj"],
        lora_alpha=cfg.lora_alpha,
        lora_dropout=cfg.lora_dropout,
        bias="none",
        use_gradient_checkpointing=False,
        random_state=cfg.global_seed,
    )

    # -------------------------
    # 4) Reward functions (correctness + format penalty + distance shaping)
    # -------------------------
    def _trim_to_stop(text: str, stop_string: str) -> str:
        if not stop_string:
            return text
        idx = text.find(stop_string)
        if idx == -1:
            return text
        return text[: idx + len(stop_string)]

    _ANS_RE = re.compile(r"<answer>\s*([-+]?\d+)\s*</answer>", flags=re.IGNORECASE | re.DOTALL)

    def _extract_int_from_answer(text: str) -> Optional[int]:
        m = _ANS_RE.search(text)
        if not m:
            return None
        try:
            return int(m.group(1))
        except Exception:
            return None

    # --- Reward functions ---
    # TRL/GRPO может передавать gold-ответы под разными ключами (answer/answers/labels...),
    # поэтому делаем функцию-адаптер, чтобы не ловить TypeError/KeyError.
    _VERIFY_NARGS = None
    try:
        _VERIFY_NARGS = len(inspect.signature(env.verify).parameters)
    except Exception:
        _VERIFY_NARGS = None

    def _verify(prompt: str, completion: str, answer: str) -> bool:
        """Совместимость с разными реализациями env.verify:
        - env.verify(prompt, completion, answer)
        - env.verify(Data(...), completion)  (как в этом ноутбуке)
        """
        if _VERIFY_NARGS == 3:
            return bool(env.verify(prompt, completion, answer))
        return bool(env.verify(Data(question=str(prompt), answer=str(answer)), completion))

    def _get_answers(answers, kwargs, n: int):
        if answers is not None:
            return answers
        for k in ("answers", "answer", "labels", "label", "solutions", "solution"):
            if k in kwargs and kwargs[k] is not None:
                return kwargs[k]
        return [""] * int(n)

    def correctness_reward_func(prompts, completions, answers=None, **kwargs):
        answers = _get_answers(answers, kwargs, len(completions))
        out = []
        for p, c, a in zip(prompts, completions, answers):
            c_use = _trim_to_stop(c, cfg.stop_string) if cfg.stop_on_answer_tag else c
            ok = _verify(str(p), str(c_use), str(a))
            out.append(1.0 if ok else 0.0)
        return out

    def format_penalty_reward_func(prompts, completions, answers=None, **kwargs):
        # answers не нужны, но оставляем аргумент для совместимости
        out = []
        for c in completions:
            c_use = _trim_to_stop(c, cfg.stop_string) if cfg.stop_on_answer_tag else c
            pred = _extract_int_from_answer(c_use)
            out.append(0.0 if pred is not None else float(cfg.format_penalty))
        return out

    def distance_reward_func(prompts, completions, answers=None, **kwargs):
        answers = _get_answers(answers, kwargs, len(completions))
        w = float(cfg.distance_reward_weight)
        if w <= 0:
            return [0.0 for _ in completions]
        out = []
        for c, a in zip(completions, answers):
            c_use = _trim_to_stop(c, cfg.stop_string) if cfg.stop_on_answer_tag else c
            pred = _extract_int_from_answer(c_use)
            try:
                gold = int(a)
            except Exception:
                gold = None
            if pred is None or gold is None:
                out.append(0.0)
                continue
            dist = abs(int(pred) - int(gold))
            out.append(w * math.exp(-dist))
        return out

    reward_funcs = [correctness_reward_func, format_penalty_reward_func]
    if cfg.distance_reward_weight > 0:
        reward_funcs.append(distance_reward_func)

    # -------------------------
    # 5) Dev-eval helpers
    # -------------------------
    def _gen_kwargs(max_new_tokens: int, temperature: float):
        d = dict(
            max_new_tokens=int(max_new_tokens),
            do_sample=(temperature > 0),
        )
        if temperature and temperature > 0:
            d["temperature"] = float(temperature)
        return d

    def _eval_on_subset(dataset_rows: List[Dict[str, Any]], indices: List[int]) -> float:
        correct = 0
        total = len(indices)

        # Unsloth: быстрее в inference
        try:
            if hasattr(model, "for_inference"):
                model.for_inference()
        except Exception:
            pass

        for i in indices:
            ex = dataset_rows[i]
            user_prompt = ex["prompt"]
            gold = ex["answer"]

            prompt_text = build_chat_prompt(tokenizer, user_prompt)
            inputs = tokenizer(
                prompt_text,
                return_tensors="pt",
                truncation=True,
                max_length=cfg.max_prompt_length,
            ).to(model.device)

            with torch.no_grad():
                out = model.generate(**inputs, **_gen_kwargs(cfg.dev_max_new_tokens, cfg.dev_temperature))

            gen_ids = out[0][inputs["input_ids"].shape[1]:]
            completion = tokenizer.decode(gen_ids, skip_special_tokens=True)

            if cfg.stop_on_answer_tag:
                completion = _trim_to_stop(completion, cfg.stop_string)

            if _verify(user_prompt, completion, gold):
                correct += 1

        # обратно в training режим
        try:
            if hasattr(model, "for_training"):
                model.for_training()
        except Exception:
            pass

        return correct / max(total, 1)

    def _eval_full(dataset_rows: List[Dict[str, Any]]) -> float:
        return _eval_on_subset(dataset_rows, list(range(len(dataset_rows))))

    def _compute_primary_metric(accs: Dict[str, float], best_metric: str, weights: Dict[str, float]) -> float:
        if best_metric.startswith("dev/") and best_metric != "dev/avg":
            key = best_metric.split("/", 1)[1]
            return float(accs.get(key, 0.0))
        # weighted avg
        num = 0.0
        den = 0.0
        for k, w in (weights or {}).items():
            if k in accs:
                num += float(w) * float(accs[k])
                den += float(w)
        if den <= 0:
            return float(np.mean(list(accs.values()))) if accs else 0.0
        return num / den

    def _capture_lora_state_to_cpu() -> Dict[str, torch.Tensor]:
        st = {}
        for n, p in model.named_parameters():
            if p.requires_grad:
                st[n] = p.detach().cpu()
        return st

    # -------------------------
    # 6) Curriculum phases
    # -------------------------
    if cfg.curriculum_enabled and cfg.curriculum_phases:
        phases = cfg.curriculum_phases
    else:
        phases = [{
            "name": "single",
            "min_difficulty": cfg.train_min_difficulty,
            "max_difficulty": cfg.train_max_difficulty,
            "train_size": cfg.train_size,
            "steps": cfg.max_steps,
            "seed_offset": 0,
            "learning_rate": cfg.learning_rate,
            "kl_beta": cfg.kl_beta,
            "dev_eval_every_steps": cfg.dev_eval_every_steps,
            "dev_eval_n": cfg.dev_eval_n,
            "max_completion_length": cfg.max_completion_length,
        }]

    start_phase_idx = int(getattr(cfg, "start_phase_idx", 0) or 0)
    start_phase_idx = max(0, min(start_phase_idx, len(phases)-1))

    # глобальный best (по выбранной dev-метрике)
    best = {
        "score": -1.0,
        "accs": None,
        "step": None,
        "phase": None,
        "lora_state": None,
    }

    phase_ckpt_dir = Path(cfg.phase_checkpoints_dir)
    phase_ckpt_dir.mkdir(parents=True, exist_ok=True)

    # -------------------------
    # 7) Семплинг trainset под фазу (+ replay)
    # -------------------------
    def _train_path(phase_name: str, min_d: int, max_d: int, n: int, seed: int) -> Path:
        return DATA_DIR / f"train_{phase_name}_d{min_d}-{max_d}_n{n}_seed{seed}.jsonl"

    def _sample_rows(n: int, min_d: int, max_d: int, seed: int) -> List[Dict[str, Any]]:
        gen = training_generator(env, min_difficulty=min_d, max_difficulty=max_d, seed=seed)
        rows = []
        for _ in range(n):
            item = next(gen)
            q = item.get("question") or item.get("prompt") or ""
            a = item.get("answer") or ""
            rows.append({"prompt": build_chat_prompt(tokenizer, str(q)), "answer": str(a)})
        return rows

    def _make_or_load_phase_trainset(phase: Dict[str, Any]) -> Dataset:
        phase_name = str(phase["name"])
        min_d = int(phase["min_difficulty"])
        max_d = int(phase["max_difficulty"])
        n = int(phase.get("train_size", cfg.train_size))
        seed = int(cfg.train_seed + 1337 * int(phase.get("seed_offset", 0)))

        path = _train_path(phase_name, min_d, max_d, n, seed)

        # если нужен replay — делаем датасет "main + replay", потом shuffle
        replay_frac = float(phase.get("replay_fraction", 0.0) or 0.0)
        replay_min = int(phase.get("replay_min_difficulty", cfg.train_min_difficulty))
        replay_max = int(phase.get("replay_max_difficulty", min_d - 1)) if phase.get("replay_max_difficulty") is None else int(phase.get("replay_max_difficulty", min_d - 1))

        # IMPORTANT: чтобы кэш trainset не переиспользовался при смене replay_*,
        # добавляем replay-параметры в имя файла.
        if replay_frac > 0 and replay_min <= replay_max:
            rp = int(round(replay_frac * 100))
            path = path.with_name(f"{path.stem}_replay{rp}_d{replay_min}-{replay_max}{path.suffix}")


        if path.exists() and (not cfg.recreate_train) and (not phase.get("recreate_train", False)):
            rows = load_jsonl(path)
            return Dataset.from_list(rows)

        if replay_frac > 0 and replay_min <= replay_max:
            n_replay = int(round(n * replay_frac))
            n_main = max(0, n - n_replay)

            rows_main = _sample_rows(n_main, min_d, max_d, seed=seed)
            rows_replay = _sample_rows(n_replay, replay_min, replay_max, seed=seed + 424242)

            rows = rows_main + rows_replay
            rng = random.Random(seed + 777)
            rng.shuffle(rows)
        else:
            rows = _sample_rows(n, min_d, max_d, seed=seed)

        save_jsonl(rows, path)
        return Dataset.from_list(rows)

    # -------------------------
    # 8) Dev-eval callback (+ early stop) + phase-best
    # -------------------------
    from transformers import TrainerCallback

    class DevEvalCallback(TrainerCallback):
        def __init__(self, phase: Dict[str, Any], global_step_offset: int):
            self.phase = phase
            self.global_step_offset = global_step_offset

            self.phase_name = str(phase["name"])
            self.best_metric = str(phase.get("best_metric", cfg.best_metric))
            self.weights = dict(phase.get("dev_metric_weights", cfg.dev_metric_weights) or {})

            self.eval_every = int(phase.get("dev_eval_every_steps", cfg.dev_eval_every_steps))
            self.quick_k = int(phase.get("dev_eval_n", cfg.dev_eval_n))

            # early stop params
            self.early_stop_enabled = bool(cfg.early_stop_enabled and phase.get("early_stop_enabled", True))
            self.patience = int(phase.get("early_stop_patience_evals", cfg.early_stop_patience_evals))
            self.warmup = int(phase.get("early_stop_warmup_evals", cfg.early_stop_warmup_evals))
            self.min_delta = float(phase.get("best_min_delta", cfg.best_min_delta))

            self.eval_calls = 0
            self.no_improve = 0
            self.best_seen = -1.0

            # phase-best
            self.phase_best_score = -1.0
            self.phase_best_step_abs = None
            self.phase_best_accs = None
            self.phase_best_lora_state = None

        def on_step_end(self, args, state, control, **kwargs):
            nonlocal best

            local_step = int(state.global_step)
            if local_step <= 0:
                return control
            if (local_step % self.eval_every) != 0:
                return control

            self.eval_calls += 1
            abs_step = self.global_step_offset + local_step

            # ------------------------------------------------------------
            # КЛЮЧ: во 2-й фазе всегда считаем FULL (без quick)
            # Можно переопределить явно: self.force_full_eval = True
            # ------------------------------------------------------------
            force_full_eval = bool(getattr(self, "force_full_eval", False)) or str(self.phase_name).startswith("phase2")

            # 1) Вычисляем accs + primary
            if force_full_eval:
                # FULL по всем devset'ам
                accs = {name: _eval_full(dev_sets[name]) for name in dev_sets.keys()}
                primary = _compute_primary_metric(accs, self.best_metric, self.weights)

                nice = " | ".join([f"{k}={accs[k]:.4f}" for k in sorted(accs.keys())])
                # n для каждого devset (обычно одинаковое, но пусть будет явно)
                n_info = ", ".join([f"{k}:{len(dev_sets[k])}" for k in sorted(dev_sets.keys())])
                print(f"[DEV-FULL  @ step {abs_step} | {self.phase_name}] {nice} | {self.best_metric}={primary:.4f} (n={n_info})")

                if experiment is not None:
                    try:
                        experiment.log_metric(f"dev_full/{self.phase_name}/{self.best_metric}", float(primary), step=abs_step)
                        for k, v in accs.items():
                            experiment.log_metric(f"dev_full/{self.phase_name}/{k}", float(v), step=abs_step)
                    except Exception:
                        pass

            else:
                # QUICK (как было раньше) — для фаз, где хотим экономить время
                accs = {}
                for spec in cfg.dev_specs:
                    name = str(spec["name"])
                    idxs = dev_quick_indices[name][: self.quick_k]
                    accs[name] = _eval_on_subset(dev_sets[name], idxs)

                primary = _compute_primary_metric(accs, self.best_metric, self.weights)

                nice = " | ".join([f"{k}={accs[k]:.4f}" for k in sorted(accs.keys())])
                print(f"[DEV-QUICK @ step {abs_step} | {self.phase_name}] {nice} | {self.best_metric}={primary:.4f} (n={self.quick_k} each)")

                if experiment is not None:
                    try:
                        experiment.log_metric(f"dev_quick/{self.phase_name}/{self.best_metric}", float(primary), step=abs_step)
                        for k, v in accs.items():
                            experiment.log_metric(f"dev_quick/{self.phase_name}/{k}", float(v), step=abs_step)
                    except Exception:
                        pass

            # 2) Phase-best (для resume/перехода между фазами) — теперь:
            #    - в фазе2 будет по FULL (потому что primary = FULL)
            if primary > (self.phase_best_score + self.min_delta):
                self.phase_best_score = float(primary)
                self.phase_best_step_abs = int(abs_step)
                self.phase_best_accs = dict(accs)
                self.phase_best_lora_state = _capture_lora_state_to_cpu()

                if getattr(cfg, "save_phase_best", True):
                    phase_ckpt_dir.mkdir(parents=True, exist_ok=True)
                    phase_best_path = phase_ckpt_dir / f"{phase_name}_best_lora.pt"
                    torch.save(self.phase_best_lora_state, phase_best_path)

                    phase_best_meta = {
                        "phase": phase_name,
                        "score": float(self.phase_best_score),
                        "step_abs": int(self.phase_best_step_abs),
                        "step_phase": int(state.global_step),
                        "checkpoint_dir": str(Path(args.output_dir) / f"checkpoint-{state.global_step}"),
                        "accs": {k: float(v) for k, v in self.phase_best_accs.items()},
                        "lora_path": str(phase_best_path),
                    }
                    save_json(phase_best_meta, phase_ckpt_dir / f"{phase_name}_best_meta.json")

                # сохранить полноценный trainer checkpoint (optimizer/scheduler/state)
                control.should_save = True

            # 3) Global-best
            def _maybe_update_global(score: float, accs_full: Dict[str, float]):
                nonlocal best
                if score > (best["score"] + self.min_delta):
                    print(f"[BEST] {self.best_metric}={score:.4f} @ step={abs_step} (phase={self.phase_name})")
                    best["score"] = float(score)
                    best["accs"] = dict(accs_full)
                    best["step"] = int(abs_step)
                    best["phase"] = self.phase_name
                    best["lora_state"] = _capture_lora_state_to_cpu()

            if cfg.select_best_by_dev and (primary > (best["score"] + self.min_delta)):
                if force_full_eval:
                    # уже FULL — подтверждать не нужно
                    _maybe_update_global(primary, accs)
                else:
                    # прежняя логика: quick → при улучшении можно подтвердить full
                    if cfg.dev_full_eval_on_best:
                        full_accs = {name: _eval_full(dev_sets[name]) for name in dev_sets.keys()}
                        full_primary = _compute_primary_metric(full_accs, self.best_metric, self.weights)
                        nice2 = " | ".join([f"{k}={full_accs[k]:.4f}" for k in sorted(full_accs.keys())])
                        print(f"[DEV-FULL  @ step {abs_step} | {self.phase_name}] {nice2} | {self.best_metric}={full_primary:.4f}")

                        if experiment is not None:
                            try:
                                experiment.log_metric(f"dev_full/{self.phase_name}/{self.best_metric}", float(full_primary), step=abs_step)
                                for k, v in full_accs.items():
                                    experiment.log_metric(f"dev_full/{self.phase_name}/{k}", float(v), step=abs_step)
                            except Exception:
                                pass

                        _maybe_update_global(full_primary, full_accs)
                    else:
                        _maybe_update_global(primary, accs)

            # 4) Early stop — базируемся на primary (во 2-й фазе это FULL)
            improved = primary > (self.best_seen + self.min_delta)
            if improved:
                self.best_seen = float(primary)
                self.no_improve = 0
            else:
                self.no_improve += 1

            if self.early_stop_enabled and (self.eval_calls > self.warmup):
                if self.no_improve >= self.patience:
                    print(
                        f"[EARLY-STOP] phase={self.phase_name}: no improvement for {self.no_improve} evals "
                        f"(patience={self.patience}). Stopping training for this phase."
                    )
                    control.should_training_stop = True

            return control

    # -------------------------
    # 9) Тренировка по фазам
    # -------------------------
    global_step_offset = 0

    for phase_idx in range(start_phase_idx, len(phases)):
        phase = phases[phase_idx]
        phase_name = str(phase["name"])
        print(f"\n=== START {phase_name} (phase_idx={phase_idx}) ===")

        # 9.1) Если стартуем не с первой фазы — пробуем загрузить best предыдущей фазы
        if cfg.resume_from_previous_phase_best and phase_idx > 0:
            prev_name = str(phases[phase_idx - 1]["name"])
            prev_best_path = phase_ckpt_dir / f"{prev_name}_best_lora.pt"
            if prev_best_path.exists():
                try:
                    state = torch.load(prev_best_path, map_location="cpu")
                    model.load_state_dict(state, strict=False)
                    print(f"[RESUME] Loaded previous phase best LoRA: {prev_best_path}")
                except Exception as e:
                    print(f"[RESUME] Failed to load {prev_best_path}: {repr(e)}")
            else:
                if phase_idx == start_phase_idx and start_phase_idx > 0:
                    raise FileNotFoundError(
                        f"start_phase_idx={start_phase_idx}, но нет файла {prev_best_path}. "
                        f"Либо запусти фазу1 (start_phase_idx=0), либо положи чекпоинт фазы1."
                    )

        # 9.2) Trainset под фазу
        train_ds = _make_or_load_phase_trainset(phase)

        # 9.3) Гиперпараметры фазы
        lr = float(phase.get("learning_rate", cfg.learning_rate))
        beta = float(phase.get("kl_beta", cfg.kl_beta))
        max_steps = int(phase.get("steps", cfg.max_steps))
        max_comp = int(phase.get("max_completion_length", cfg.max_completion_length))

        # 9.4) GRPOConfig с аккуратным "best-effort" для разных версий trl
        def _grpo_extra_args():
            extra = {}
            sig = inspect.signature(GRPOConfig.__init__)
            params = sig.parameters

            # stop sequences / stop strings (если поддерживается)
            for key in ["stop_strings", "stop_sequences", "stop_sequence", "stop"]:
                if key in params and cfg.stop_on_answer_tag and cfg.stop_string:
                    extra[key] = [cfg.stop_string] if ("strings" in key or "sequences" in key) else cfg.stop_string

            # иногда параметр называется "beta" или "kl_beta" — trl версии разные
            if "beta" in params:
                extra["beta"] = beta
            if "kl_beta" in params:
                extra["kl_beta"] = beta

            return extra

        grpo_args = GRPOConfig(
            output_dir=str(RESULTS_DIR / cfg.grpo_output_dir / phase_name),
            learning_rate=lr,
            per_device_train_batch_size=cfg.per_device_train_batch_size,
            gradient_accumulation_steps=cfg.gradient_accumulation_steps,
            num_generations=cfg.num_generations,
            max_prompt_length=cfg.max_prompt_length,
            max_completion_length=max_comp,
            logging_steps=cfg.logging_steps,
            save_strategy="steps",
            save_steps=cfg.save_steps,
            save_total_limit=getattr(cfg, "save_total_limit", None),
            max_steps=max_steps,
            **_grpo_extra_args(),
        )

        # 9.5) Trainer
        callback = DevEvalCallback(phase=phase, global_step_offset=global_step_offset)

        trainer = GRPOTrainer(
            model=model,
            args=grpo_args,
            train_dataset=train_ds,
            reward_funcs=reward_funcs,
            callbacks=[callback],
        )

        # 9.6) train (можно продолжать с чекпоинта)
        resume_ckpt = phase.get("resume_from_checkpoint", None) or getattr(cfg, "resume_from_checkpoint", None)
        trainer.train(resume_from_checkpoint=resume_ckpt)
        print(f"=== END {phase_name} ===\n")

        # 9.7) сохранить BEST LoRA текущей фазы (именно лучшую точку внутри фазы)
        if cfg.save_phase_best:
            phase_best_path = phase_ckpt_dir / f"{phase_name}_best_lora.pt"
            state_to_save = callback.phase_best_lora_state or _capture_lora_state_to_cpu()
            torch.save(state_to_save, phase_best_path)
            print(f"[SAVED] Phase best LoRA -> {phase_best_path} (best_score={callback.phase_best_score:.4f}, step={callback.phase_best_step_abs})")

        global_step_offset += int(trainer.state.global_step)

        # чистим память
        try:
            cleanup_cuda()
        except Exception:
            pass

    # -------------------------
    # 10) Применяем глобальный best LoRA и сохраняем merged модель
    # -------------------------
    if best.get("lora_state") is not None:
        model.load_state_dict(best["lora_state"], strict=False)
        print(f"[LOAD] Applied GLOBAL best LoRA from phase={best.get('phase')} step={best.get('step')} score={best.get('score'):.4f}")

    merged_dir = MODELS_DIR / cfg.merged_model_subdir
    merged_dir.mkdir(parents=True, exist_ok=True)

    model.save_pretrained_merged(str(merged_dir), tokenizer, save_method="merged_16bit")
    tokenizer.save_pretrained(str(merged_dir))

    # best содержит тензоры (LoRA state) -> для JSON сохраняем только метаданные
    best_meta = {k: v for k, v in best.items() if k != "lora_state"}
    if isinstance(best_meta.get("score"), (int, float)):
        best_meta["score"] = float(best_meta["score"])
    if isinstance(best_meta.get("step"), (int, float)):
        best_meta["step"] = int(best_meta["step"])
    if isinstance(best_meta.get("accs"), dict):
        best_meta["accs"] = {k: float(v) for k, v in best_meta["accs"].items()}
    save_json(best_meta, RESULTS_DIR / "best_model_by_dev.json")

    print("[DONE] merged model saved to:", merged_dir.resolve())
    return str(merged_dir)


In [13]:
# Запуск GRPO-обучения (может занять время; на Colab/Kaggle нужен GPU)
TRAINED_MODEL_DIR = run_train(cfg, env)

# Для удобства: сохраняем путь к модели как артефакт эксперимента
save_json({"trained_model_dir": TRAINED_MODEL_DIR}, RESULTS_DIR / "trained_model.json")

TRAINED_MODEL_DIR


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.1: Fast Qwen2 patching. Transformers: 4.57.2.
   \\   /|    NVIDIA RTX PRO 6000 Blackwell Workstation Edition. Num GPUs = 1. Max memory: 95.592 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 12.0. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.35. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.11.1 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.



=== START phase1_d1-5 (phase_idx=0) ===
Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 8


The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 8,000 | Num Epochs = 1 | Total steps = 200
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 8 x 1) = 64
 "-____-"     Trainable parameters = 18,464,768 of 1,562,179,072 (1.18% trained)
COMET INFO: An experiment with the same configuration options is already running and will be reused.
COMET WARNING: String value length exceeds 1000 characters and will be truncated. Provided value: 'LoraConfig(task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, peft_type=<PeftType.LORA: 'LORA'>, auto_mapping={'base_model_class': 'Qwen2ForCausalLM', 'parent_library': 'transformers.models.qwen2.modeling_qwen2', 'unsloth_fixed': True}, peft_version='0.18.1', base_model_name_or_path='unsloth/qwen2.5-1.5b-instruct-unsloth-bnb-4bit', revision=No

Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,sampling / sampling_logp_difference / mean,sampling / sampling_logp_difference / max,sampling / importance_sampling_ratio / min,sampling / importance_sampling_ratio / mean,sampling / importance_sampling_ratio / max,kl,rewards / correctness_reward_func / mean,rewards / correctness_reward_func / std,rewards / format_penalty_reward_func / mean,rewards / format_penalty_reward_func / std,rewards / distance_reward_func / mean,rewards / distance_reward_func / std
10,0.000000,0.044623,0.246385,40.929688,3.100000,64.000000,0.535937,14.378618,3.100000,55.700000,0,0,0,0,0,0.001757,0.103125,0.297771,-0.064063,0.048062,0.005560,0.012173
20,0.000100,0.011978,0.188831,41.721875,3.100000,64.000000,0.556250,13.803588,3.100000,52.900000,No Log,No Log,No Log,No Log,No Log,0.006445,0.070312,0.236161,-0.063906,0.048290,0.005572,0.011927
30,0.001700,0.071617,0.251008,30.567187,2.200000,64.000000,0.351562,12.461327,2.200000,53.400000,No Log,No Log,No Log,No Log,No Log,0.085375,0.107813,0.264135,-0.044844,0.049348,0.008648,0.013418
40,0.004800,0.136414,0.305708,19.428125,3.300000,64.000000,0.171875,10.170739,3.300000,47.700000,No Log,No Log,No Log,No Log,No Log,0.241382,0.146875,0.351412,-0.023750,0.042723,0.013289,0.015972
50,0.006600,0.217163,0.345684,14.201562,6.200000,64.000000,0.076563,10.076723,6.200000,41.900000,No Log,No Log,No Log,No Log,No Log,0.329124,0.209375,0.404953,-0.010469,0.029757,0.018257,0.017303
60,0.007900,0.317380,0.363969,12.443750,8.000000,64.000000,0.043750,10.085685,8.000000,47.200000,No Log,No Log,No Log,No Log,No Log,0.392845,0.298438,0.456220,-0.004531,0.020598,0.023474,0.018222
70,0.008200,0.250942,0.286177,11.051562,6.200000,64.000000,0.028125,9.520852,6.200000,31.400000,No Log,No Log,No Log,No Log,No Log,0.409528,0.234375,0.419797,-0.004531,0.019120,0.021098,0.016836
80,0.010900,0.338971,0.325714,10.425000,8.000000,46.900000,0.017188,9.490553,8.000000,34.800000,No Log,No Log,No Log,No Log,No Log,0.544996,0.317188,0.450067,-0.002969,0.015779,0.024752,0.017444
90,0.008100,0.312782,0.356410,11.195312,8.600000,56.900000,0.028125,9.663995,8.600000,28.200000,No Log,No Log,No Log,No Log,No Log,0.406645,0.292187,0.450081,-0.003906,0.018047,0.024501,0.017123
100,0.016600,0.422402,0.402324,10.403125,8.600000,43.700000,0.017188,9.469310,8.600000,29.500000,No Log,No Log,No Log,No Log,No Log,0.831954,0.396875,0.483838,-0.002344,0.011264,0.027871,0.018250


...

Unsloth: Preparing safetensor model files: 100%|██████████| 1/1 [00:00<00:00, 3366.22it/s]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [00:03<00:00,  3.14s/it]


Unsloth: Merge process complete. Saved to `/home/yaros/DS-Mag/AI-SelectedTopics/W2-1/models/qwen2p5_1p5b_grpo_lis_merged`
[DONE] merged model saved to: /home/yaros/DS-Mag/AI-SelectedTopics/W2-1/models/qwen2p5_1p5b_grpo_lis_merged


'models/qwen2p5_1p5b_grpo_lis_merged'

---

## Дальше: инференс и финальная оценка

1) Активируйте **eval‑среду** (отдельный env) с `vllm` и `transformers`.

2) Откройте ноутбук **`week2_grpo_lis_infer_eval.ipynb`**.

Он автоматически:
- загрузит `results/config.json` (если он существует),
- загрузит путь к модели из `results/trained_model.json`,
- посчитает accuracy на фиксированных `data/test_*.jsonl`,
- построит график baseline vs trained.


In [14]:
# (опционально) Завершить эксперимент Comet, чтобы гарантированно "дослать" логи.
# В интерактивной работе можно не вызывать end() — Comet обычно сам всё отправит при завершении ядра.
if globals().get("COMET_EXPERIMENT", None) is not None:
    try:
        COMET_EXPERIMENT.end()
        print("[COMET] Experiment ended.")
    except Exception as e:
        print("[COMET] end() failed:", repr(e))


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : Qwen2.5-1.5B-Instruct_seed42
COMET INFO:     url                   : https://www.comet.com/yaroslav-pankratov/ds-mag-w2-grpo/58621e6aa3854b9d8267f0cf7a15daef
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     dev_full/phase1_d1-5/dev/avg [4]                   : (0.37031250000000004, 0.38125000000000003)
COMET INFO:     dev_full/phase1_d1-5/easy [4]                      : (0.4140625, 0.4453125)
COMET INFO:     dev_full/phase1_d1-5/hard [4]                      : (0.171875, 0.2109375)
COMET INFO:     dev_full/phase1_d1-5/medium [4]                    : (0.375, 0.390625)
COMET INFO:     dev_full/phase2_d6-10/dev/avg [20]                 : (0.3109

[COMET] Experiment ended.
